In [1]:
import earthchem
earthchem.__version__

'0.0.3+1.gbab5996'

Make a rest client

In [2]:
q = earthchem.Query(author='klump')
q.url

'http://ecp.iedadata.org/restsearchservice?outputtype=json&author=klump'

How many records have we found?

In [3]:
q.count()

0

In [4]:
q.dataframe()

Didn't find any records, returning None


Let's try another author

In [5]:
q = earthchem.Query(author='barnes')
q.count()

4902

Get the first 50 records

In [6]:
df = q.dataframe()
df.head()

,al2o3,author,cao,cl,composition,feot,journal,k,k2o,latitude,...,mno,na2o,p2o5,rock_name,sample_id,sio2,source,tio2,title,type
0,NaN,"BARNES, J. D.; STRAUB, S. M.",NaN,647.0,unknown,NaN,CHEM. GEOL.,NaN,NaN,18.0448,...,NaN,NaN,NaN,not-given,64DS,NaN,GEOROC,NaN,CHLORINE STABLE ISOTOPE VARIATIONS IN IZU BONI...,volcanic
1,13.3,"BARNES, J. D.; STRAUB, S. M.",9.37,560.0,unknown,12.63,CHEM. GEOL.,NaN,0.30,30.8610,...,0.31,2.20,0.10,not-given,125-782A-29X3-42-46,48.47,GEOROC,1.02,CHLORINE STABLE ISOTOPE VARIATIONS IN IZU BONI...,volcanic
2,14.2,"BARNES, J. D.; STRAUB, S. M.",8.84,960.0,unknown,10.78,CHEM. GEOL.,NaN,0.33,30.8610,...,0.38,2.60,0.10,not-given,125-782A-29X3-42-46,52.00,GEOROC,0.86,CHLORINE STABLE ISOTOPE VARIATIONS IN IZU BONI...,volcanic
3,13.6,"BARNES, J. D.; STRAUB, S. M.",9.30,1170.0,unknown,12.43,CHEM. GEOL.,NaN,0.26,30.8610,...,0.22,2.27,0.11,not-given,125-782A-29X3-42-46,49.72,GEOROC,0.98,CHLORINE STABLE ISOTOPE VARIATIONS IN IZU BONI...,volcanic
4,14.3,"BARNES, J. D.; STRAUB, S. M.",6.39,1870.0,unknown,8.02,CHEM. GEOL.,NaN,0.64,30.8610,...,0.22,3.77,0.15,not-given,125-782A-29X3-42-46,57.82,GEOROC,0.83,CHLORINE STABLE ISOTOPE VARIATIONS IN IZU BONI...,volcanic


Lets try and paginate this instead

In [8]:
q['searchtype'] = 'rowdata'
q['standarditems'] = 'yes'

In [11]:
q.count()

4902

In [18]:
items_per_page = 50
max_items = q.count()

In [1]:
import earthchem
from earthchem.pagination import make_pages

In [5]:
import tqdm
import requests
import pandas

from io import StringIO

In [6]:
self = earthchem.Query(author='barnes')
max_items = 124

In [7]:

# Get the list of pages we're going to use, use to set up tqdm and query
pages = make_pages(max_items or self.count())[:4]
tqdm_kwargs = {
    'desc': 'Downloading pages',
    'total': len(pages)
}

# Set basic search params
self['searchtype'] = 'rowdata'
self['standarditems'] = 'yes'

# Accumulate pages as we go
accumulator = None
for page in tqdm.tqdm(pages, **tqdm_kwargs):
    self['startrow'], self['endrow'] = page
    resp = requests.get(self.url)
    if resp.ok:
        # Create a new dataframe to add to the old one
        df = pandas.read_json(StringIO(resp.text))
        if accumulator is None:
            accumulator = df
        else:
            accumulator = pandas.concat([accumulator, df])

In [10]:
len(accumulator)

125

Now we can generate a plot of this data

In [ ]:
%matplotlib inline

df.plot('longitude', 'latitude', 'scatter')

In [ ]:
df.plot('al2o3', 'mgo', 'scatter')